# Extract Image Brightness

In [9]:
import pandas as pd

df = pd.read_csv("imageDF.csv", sep=";", header = 0)

# cast all columns of df to string
df = df.astype(str)

In [13]:
image_urls = df["image 1"]

#convert image_urls to list
image_urls = image_urls.tolist()

# get indices of the elements in image_urls which do not contain "https"
indices = [i for i, s in enumerate(image_urls) if "https" not in s]
 
# remove the elements in image_urls which do not contain "https"
for index in sorted(indices, reverse=True):
    del image_urls[index]


In [14]:
from PIL import Image, ImageStat
import requests
from io import BytesIO
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

def brightness(im_url):
   response = requests.get(im_url, headers=headers)
   if response.status_code == 200:
      image_data = response.content
      
      image = Image.open(BytesIO(image_data)).convert('L')
      stat = ImageStat.Stat(image)
      return stat.mean[0]
   else:
        return None

In [ ]:
from tqdm import tqdm
from PIL import UnidentifiedImageError

image_brightness = []
for url in tqdm(image_urls):
    try:
        out = brightness(url)
        image_brightness.append(out)
    except UnidentifiedImageError:
        image_brightness.append(None)
        continue


In [60]:
# add NaNs at indices where there are no images (based on the indices variable)
for index in indices:
    image_brightness.insert(index, "")

In [ ]:
# add image brightness to df as column

df["image brightness"] = image_brightness

In [78]:
# save the dataframe to a csv file
df.to_csv("imageDF_brightness.csv", sep=";", index=False)

In [ ]:
# in google colab download the file
#from google.colab import files
#files.download("/content/imageDF1.csv")